In [1]:
import pandas as pd
from pulp import *

In [2]:
# !pip install openpyxl

## Plant Location  

#### Manufacturing variable costs

In [3]:
# Import Costs
manvar_costs = pd.read_excel('variable_costs.xlsx', index_col = 0)
manvar_costs

,USA,Germany,Japan,Brazil,India
Variable Costs ($/Unit),,,,,
USA,12,12,12,12,12
Germany,13,13,13,13,13
Japan,10,10,10,10,10
Brazil,8,8,8,8,8
India,5,5,5,5,5


In [4]:
time_cost = pd.read_excel('time_cost.xlsx', index_col = 0)

In [5]:
time_cost

,USA,Germany,Japan,Brazil,India
Variable time (hours),,,,,
USA,0,10,8,2,32
Germany,10,0,28,8,12
Japan,8,28,0,10,14
Brazil,2,8,10,0,18
India,32,12,14,18,0


In [6]:
fresh = 30
fresh_time = time_cost.copy()
for i in fresh_time:
    for j in fresh_time:
        if i!=j:
            fresh_time[i][j] = fresh

In [7]:
fresh_time

,USA,Germany,Japan,Brazil,India
Variable time (hours),,,,,
USA,0,30,30,30,30
Germany,30,0,30,30,30
Japan,30,30,0,30,30
Brazil,30,30,30,0,30
India,30,30,30,30,0


In [8]:
expire_time = time_cost - fresh_time
expire_time

,USA,Germany,Japan,Brazil,India
Variable time (hours),,,,,
USA,0,-20,-22,-28,2
Germany,-20,0,-2,-22,-18
Japan,-22,-2,0,-20,-16
Brazil,-28,-22,-20,0,-12
India,2,-18,-16,-12,0


#### Freight costs

In [9]:
# Import Costs
freight_costs = pd.read_excel('freight_costs.xlsx', index_col = 0)
freight_costs

,USA,Germany,Japan,Brazil,India
Freight Costs ($/Container),,,,,
USA,0,12250,1100,16100,8778
Germany,13335,0,8617,20244,10073
Japan,15400,22750,0,43610,14350
Brazil,16450,22050,28000,0,29750
India,13650,15400,24500,29400,0


#### Variable Costs

In [10]:
# Variable Costs
var_cost = freight_costs/1000 + manvar_costs 

var_cost

,USA,Germany,Japan,Brazil,India
Freight Costs ($/Container),,,,,
USA,12.000,24.25,13.100,28.100,20.778
Germany,26.335,13.00,21.617,33.244,23.073
Japan,25.400,32.75,10.000,53.610,24.350
Brazil,24.450,30.05,36.000,8.000,37.750
India,18.650,20.40,29.500,34.400,5.000


#### Fixed Costs

In [11]:
# Import Costs
fixed_costs = pd.read_excel('fixed_cost.xlsx', index_col = 0)
fixed_costs

,Low,High
USA,6500,9500
Germany,4980,7270
Japan,6230,9100
Brazil,3230,4730
India,2110,6160


#### Plants Capacity

In [12]:
# Two types of plants: Low Capacity and High Capacity Plant
cap = pd.read_excel('capacity.xlsx', index_col = 0)
cap

,Low,High
Capacity (kUnits/month),,
USA,500,1500
Germany,500,1500
Japan,500,1500
Brazil,500,1500
India,500,3000


#### Demand 

In [13]:
# -- Demand
demand = pd.read_excel('demand_3.xlsx', index_col = 0)
demand

,Demand
(Units/month),
USA,2800000
Germany,550000
Japan,0
Brazil,0
India,239000


In [14]:
# Define Decision Variables
loc = ['USA', 'Germany', 'Japan', 'Brazil', 'India']
size = ['Low', 'High']

# Initialize Class
model = LpProblem("Capacitated Plant Location Model", LpMinimize)


# Create Decision Variables
x = LpVariable.dicts("production_", [(i,j) for i in loc for j in loc],
                     lowBound=0, upBound=None, cat='continuous')
y = LpVariable.dicts("plant_", 
                     [(i,s) for s in size for i in loc], cat='Binary')
# z = LpVariable.dicts("time_", [(i,j) for i in loc for j in loc],
#                      lowBound=0, upBound=None)

# Define Objective Function
model += (lpSum([fixed_costs.loc[i,s] * y[(i,s)] * 1000 for s in size for i in loc])
          + lpSum([var_cost.loc[i,j] * x[(i,j)]   for i in loc for j in loc]))

# Add Constraints
for j in loc:
    model += lpSum([x[(i, j)] for i in loc]) == demand.loc[j,'Demand']
for i in loc:
    model += lpSum([x[(i, j)] for j in loc]) <= lpSum([cap.loc[i,s]*y[(i,s)] * 1000
                                                       for s in size])
for i in loc:
    for j in loc:
        model+= lpSum(x[(i, j)]*time_cost.loc[i, j]) <= lpSum(fresh*x[(i, j)])
    #time_cost


# Define logical constraint: Add a logical constraint so that if the high capacity plant in USA is open, then a low capacity plant in Germany is also opened.
# model += y[('USA','High_Cap')] <= y[('Germany','Low_Cap')]                                                       
                                                       
# Solve Model
model.solve()
print("Total Costs = {:,} ($/Month)".format(int(value(model.objective))))
print('\n' + "Status: {}".format(LpStatus[model.status]))


# Dictionnary
dict_plant = {}
dict_prod = {}
for v in model.variables():
    if 'plant' in v.name:
        name = v.name.replace('plant__', '').replace('_', '')
        dict_plant[name] = int(v.varValue)
        p_name = name
    else:
        name = v.name.replace('production__', '').replace('_', '')
        dict_prod[name] = v.varValue
    print(name, "=", v.varValue)
    


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/animesh/.local/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/e780f4e36b21445bbaa646bd3c534875-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/e780f4e36b21445bbaa646bd3c534875-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 40 COLUMNS
At line 181 RHS
At line 217 BOUNDS
At line 228 ENDATA
Problem MODEL has 35 rows, 35 columns and 85 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 7.35841e+07 - 0.00 seconds
Cgl0004I processed model has 8 rows, 23 columns (10 integer (10 of which binary)) and 36 elements
Cbc0038I Initial state - 3 integers unsatisfied sum - 1.31133
Cbc0038I Pass   1: suminf.    0.05933 (1) obj. 9.18193e+07 iterations 6
Cbc0038I Solution found of 9.86579e+07
Cbc0038I Relaxing continuous gives 8.42244e+07
Cbc0038I Before mini branch and

/home/animesh/.local/lib/python3.8/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [15]:
# Capacity Plant
list_low, list_high = [], []
for l in loc:
    for cap in ['Low', 'High']:
        x = "('{}','{}')".format(l, cap)
        if cap == 'Low':
            list_low.append(dict_plant[x])
        else:
            list_high.append(dict_plant[x])
df_capacity = pd.DataFrame({'Location': loc, 'Low': list_low, 'High':
                            list_high}).set_index('Location')
    
df_capacity

,Low,High
Location,,
USA,1,1
Germany,0,1
Japan,0,0
Brazil,0,0
India,1,0


In [16]:
dict_plant

{"('Brazil','High')": 0,
 "('Brazil','Low')": 0,
 "('Germany','High')": 1,
 "('Germany','Low')": 0,
 "('India','High')": 0,
 "('India','Low')": 1,
 "('Japan','High')": 0,
 "('Japan','Low')": 0,
 "('USA','High')": 1,
 "('USA','Low')": 1}

In [17]:
model.variables()

[plant__('Brazil',_'High'),
 plant__('Brazil',_'Low'),
 plant__('Germany',_'High'),
 plant__('Germany',_'Low'),
 plant__('India',_'High'),
 plant__('India',_'Low'),
 plant__('Japan',_'High'),
 plant__('Japan',_'Low'),
 plant__('USA',_'High'),
 plant__('USA',_'Low'),
 production__('Brazil',_'Brazil'),
 production__('Brazil',_'Germany'),
 production__('Brazil',_'India'),
 production__('Brazil',_'Japan'),
 production__('Brazil',_'USA'),
 production__('Germany',_'Brazil'),
 production__('Germany',_'Germany'),
 production__('Germany',_'India'),
 production__('Germany',_'Japan'),
 production__('Germany',_'USA'),
 production__('India',_'Brazil'),
 production__('India',_'Germany'),
 production__('India',_'India'),
 production__('India',_'Japan'),
 production__('India',_'USA'),
 production__('Japan',_'Brazil'),
 production__('Japan',_'Germany'),
 production__('Japan',_'India'),
 production__('Japan',_'Japan'),
 production__('Japan',_'USA'),
 production__('USA',_'Brazil'),
 production__('USA',_'

In [18]:
## Total Costs = 99,406,000 ($/Month)